<a href="https://colab.research.google.com/github/Jair2472/Modelos_de_Prediccion/blob/main/LDD/An%C3%A1lisis_Profesional_de_Ventas_dpor_Dpto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import joypy # Librería para Ridgeline plots

# --- 0. Configuración Visual Profesional ---
# Usaremos un estilo consistente y legible para todos los gráficos.
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (16, 9)
plt.rcParams['axes.titlesize'] = 22
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['font.family'] = 'sans-serif'


# --- 1. Carga y Preparación de Datos (Lógica Corregida) ---
try:
    df = pd.read_csv('dataset_carniceria.csv')
except FileNotFoundError:
    print("ADVERTENCIA: No se encontró 'dataset_carniceria.csv'. Se usarán datos de ejemplo.")
    # Creamos un dataframe de ejemplo si no se encuentra el archivo.
    date_range = pd.to_datetime(pd.date_range(start='2021-01-01', periods=365*3, freq='D'))
    data = {'Date': date_range}
    df = pd.DataFrame(data)
    df['FiscalYear'] = df['Date'].dt.year
    df['VentasAcum'] = 0

    # Simulamos datos acumulados que se reinician cada año
    ventas_list = []
    for year in df['FiscalYear'].unique():
        n_days = len(df[df['FiscalYear'] == year])
        ventas_acum = (np.sin(np.arange(n_days) * 2 * np.pi / 30) * 5000 + np.sin(np.arange(n_days) * 2 * np.pi / 7) * 2000 + np.random.rand(n_days) * 1000 + 10000).cumsum()
        ventas_list.append(pd.Series(ventas_acum))

    df['VentasAcum'] = pd.concat(ventas_list).values

# Convertimos la columna 'Date' a formato datetime.
if isinstance(df['Date'].iloc[0], str):
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Ordenamos el dataframe para asegurar la secuencia correcta
df = df.sort_values(['FiscalYear', 'Date']).reset_index(drop=True)

# Lógica de cálculo de ventas diarias corregida, agrupando por año.
calc_daily = lambda x: x.diff().fillna(x.iloc[0])
df['VentasDiarias'] = df.groupby('FiscalYear')['VentasAcum'].transform(calc_daily)
df['VentasDiarias'] = df['VentasDiarias'].clip(lower=0) # Aseguramos no tener ventas negativas

# Creamos las columnas de tiempo que usaremos
df['Año'] = df['Date'].dt.year
df['Mes'] = df['Date'].dt.month
df['SemanaDelAño'] = df['Date'].dt.isocalendar().week
df['DiaSemanaNum'] = df['Date'].dt.dayofweek
dias_map = {0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 3: 'Jueves', 4: 'Viernes', 5: 'Sábado', 6: 'Domingo'}
df['DiaSemana'] = df['DiaSemanaNum'].map(dias_map)


# --- 2. Generación de Gráficos Profesionales ---

# --- GRÁFICO 1: Boxplots de Distribución de Ventas ---
fig, axes = plt.subplots(3, 1, figsize=(18, 25))
fig.suptitle('Distribución de Ventas Diarias', fontsize=30, weight='bold', y=0.95)

# Boxplot por Día de la Semana
orden_dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
sns.boxplot(ax=axes[0], data=df, x='DiaSemana', y='VentasDiarias', order=orden_dias, palette='coolwarm')
axes[0].set_title('Por Día de la Semana', fontsize=20)
axes[0].set_xlabel('Día de la Semana')
axes[0].set_ylabel('Ventas Diarias ($)')

# Boxplot por Mes
sns.boxplot(ax=axes[1], data=df, x='Mes', y='VentasDiarias', palette='viridis')
axes[1].set_title('Por Mes', fontsize=20)
axes[1].set_xlabel('Mes del Año')
axes[1].set_ylabel('Ventas Diarias ($)')

# Boxplot por Año
sns.boxplot(ax=axes[2], data=df, x='Año', y='VentasDiarias', palette='magma')
axes[2].set_title('Por Año', fontsize=20)
axes[2].set_xlabel('Año')
axes[2].set_ylabel('Ventas Diarias ($)')

plt.tight_layout(rect=[0, 0.03, 1, 0.93])
plt.savefig('grafico_profesional_1_boxplots.png')
plt.close()


# --- GRÁFICO 2: El Horizonte de las Ventas (Ridgeline Plot) ---
# Muestra la distribución de ventas para cada mes, permitiendo ver la estacionalidad y la consistencia.
fig, axes = joypy.joyplot(
    data=df[['VentasDiarias', 'Mes']],
    by='Mes',
    colormap=plt.cm.autumn_r,
    figsize=(16, 10),
    linewidth=1,
    legend=False,
    alpha=0.8,
    tails=0.1
)
plt.title('Horizonte de Ventas: Distribución Mensual', fontsize=26, weight='bold', pad=40)
plt.xlabel('Ventas Diarias ($)')
plt.savefig('grafico_profesional_2_ridgeline.png')
plt.close()


# --- GRÁFICO 3: El ADN Semanal del Año (Heatmap) ---
# Visualiza las semanas "calientes" y "frías" del año, revelando patrones estacionales y de feriados.
# Usamos el último año completo para el análisis.
ultimo_año_completo = df[df['Año'] < df['Año'].max()]['Año'].max()
df_heatmap = df[df['Año'] == ultimo_año_completo].copy()
df_heatmap = df_heatmap.pivot_table(
    values='VentasDiarias',
    index='DiaSemana',
    columns='SemanaDelAño',
    aggfunc='mean'
).reindex(orden_dias)

plt.figure(figsize=(22, 8))
sns.heatmap(df_heatmap, cmap='YlGnBu', linewidths=.5, linecolor='lightgray', robust=True)
plt.title(f'ADN de Ventas Semanales - Año {ultimo_año_completo}', fontsize=26, weight='bold', pad=20)
plt.xlabel('Semana del Año')
plt.ylabel('Día de la Semana')
plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('grafico_profesional_3_heatmap_semanal.png')
plt.close()

print("Se han generado 3 nuevos gráficos profesionales. Revisa los archivos .png.")